In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType
import pandas as pd

In [ ]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": 'CLIENT_ID',
"fs.azure.account.oauth2.client.secret": "SECRET_ID",
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/DIRECTORY_ID/oauth2/token"}

dbutils.fs.mount(
source = "abfss://CONTAINER@STORAGEACCOUNT.dfs.core.windows.net", #container@storageaccount
mount_point = "/mnt/RESOURCENAME",
extra_configs = configs)

In [ ]:
%fs
ls "/mnt/RESOURCENAME"

In [ ]:
njd2023 = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/RESOURCENAME/raw-data/njd2023.csv").toPandas()

In [ ]:
njd2023.head()

In [ ]:
team_abv = 'NJD'
season = '2023'
reg_or_playoffs = 'R'

In [ ]:
njd2023 = njd2023.loc[:, :"Opponent19"]
njd2023.drop(columns=["_c8","_c14"],inplace=True)
njd2023.head(2)

In [ ]:
njd2023.rename(columns={'_c0':'GP', '_c1':"Date", '_c2':"Home", '_c3':"Opp_Name", '_c4':"GF",
       '_c5':"GA", '_c6':"Result", '_c7':"OT", 'Team9':"Team_S", 'Team10':"Team_PIM", 'Team11':"Team_PPG",
       'Team12':"Team_PPO", 'Team13':"Team_SHG", 'Opponent15':"Opp_S", 'Opponent16':"Opp_PIM", 'Opponent17':"Opp_PPG",
       'Opponent18':"Opp_PPO", 'Opponent19':"Opp_SHG"},inplace=True)

njd2023.drop(0,inplace=True)

In [ ]:
njd2023.Home = [0 if x == "@" else 1 for x in njd2023.Home]
njd2023.OT = [1 if x == "OT" else 0 for x in njd2023.OT]

njd2023['Team_Abv'] = team_abv
njd2023['Season'] = reg_or_playoffs
njd2023['Season_Year'] = season
njd2023['Team_ID'] = 18

In [ ]:
# Create Primary Key
njd2023["Key"] = njd2023["Team_ID"].astype(str) + "-" + season + "-" + reg_or_playoffs + njd2023["GP"].astype(str)

In [ ]:
# Cleanup date
njd2023['Date'] = njd2023.Date.astype(str)
njd2023['Date'] = njd2023.Date.str.replace('-','')

In [ ]:
# Reorder columns
njd2023 = njd2023[['Key','Team_ID','Season_Year','GP','Season','Date','Home','Opp_Name','GF','GA','Result','OT','Team_S',
                   'Team_PIM', 'Team_PPG', 'Team_PPO', 'Team_SHG', 'Opp_S', 'Opp_PIM', 'Opp_PPG', 'Opp_PPO', 'Opp_SHG']]

In [ ]:
njd2023.head()

In [ ]:
njd2023.to_csv("/dbfs/mnt/RESOURCENAME/transformed-data/njd2023", header=True, index=False)